In [1]:
import numpy as np
import math
import pandas as pd
import random
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from catboost import CatBoostRegressor

from netCDF4 import Dataset

from sklearn.preprocessing import MinMaxScaler


import PCA_functions as pf
import Extreme_functions as ef

from tigramite import plotting as tp
from tigramite.pcmci import PCMCI
from tigramite.independence_tests import ParCorr, CMIknn
import tigramite.data_processing as pp

from Data import Data
from datetime import datetime

import pickle
from statsmodels.tsa.stattools import grangercausalitytests
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel

import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20,10)

Could not import rpy package
Could not import r-package RCIT


/home/mathsys2/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

def deseasonalize(data,freq=12):
    """
    The shape of data should be (time, index) 
    """
    n  = data.shape[1]
    N  = data.shape[0]
    data_deseasonal = np.zeros(data.shape)
    for i in range(n):
        temp = np.copy(data[:,i])
        r = np.zeros((N))
        for j in range(freq):
            Idx = np.arange(j,N,freq)
            if temp[Idx].std() == 0:
                r[Idx] = 0
            else:
                r[Idx] = (temp[Idx] - temp[Idx].mean())/temp[Idx].std()
        data_deseasonal[:,i] = np.copy(r)
    return(data_deseasonal)


ET_gamma = np.load("ET_gamma.npy")
N = ET_gamma.shape[0]
n_nodes = ET_gamma.shape[1]
extremes_treshold = -1
count = []
for i in range(N):
    count.append(np.count_nonzero(ET_gamma[i,:] <= extremes_treshold))

level = 12
temporal_limits = {"time_min":datetime(1977, 1, 1, 0, 0),"time_max":datetime(2015, 12, 1, 0, 0) }

In [21]:
def min_MSE_finder1(count, result_sst, link,n_estimators=100, max_depth=5, tau=-1):
    result =[]
    link = link[link[:,1] <= tau]
    refined_link = []
    df = pd.DataFrame({"drought":count, "drought1":count})
    df.drought1 = df.drought1.shift(abs(tau))
    df = df.dropna()
    index = int(df.shape[0]*0.7)
    dim = df.shape[1]

    x_train, x_test = df.iloc[:index,1:dim], df.iloc[index:,1:dim]
    y_train, y_test = df.iloc[:index,0], df.iloc[index:,0]
    model = RandomForestRegressor(max_depth=max_depth, random_state=0, n_estimators=n_estimators)
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    result.append(mean_squared_error(y_pred, y_test))


    for z in range(len(link)):
        df = pd.DataFrame({"drought":count, "drought1":count})
        df.drought1 = df.drought1.shift(abs(tau))
        for k in range(0,z+1):
                if link[k,0] != 0:
                    df[str(link[k,0]-1)] = result_sst[:,link[k,0]-1]
                    df[str(link[k,0]-1)] = df[str(link[k,0]-1)].shift(abs(link[k,1]))
                else:
                    df[str(link[k,0])] = count
                    df[str(link[k,0])] = df[str(link[k,0])].shift(abs(link[k,1]))
        df = df.dropna()
        index = int(df.shape[0]*0.7)
        dim = df.shape[1]

        x_train, x_test = df.iloc[:index,1:dim], df.iloc[index:,1:dim]
        y_train, y_test = df.iloc[:index,0], df.iloc[index:,0]
        model = RandomForestRegressor(max_depth=max_depth, random_state=0, n_estimators=n_estimators)
        model.fit(x_train, y_train)
        y_pred = model.predict(x_test)
        mse = mean_squared_error(y_pred, y_test)
        if mse < result[-1]:
            result.append(mean_squared_error(y_pred, y_test))
            refined_link.append(link[z])
    return(result,np.array(refined_link))



In [3]:
def min_MSE_finder_xgboosting(count, result_sst, link,n_estimators=100, max_depth=5, tau=-1):
    result =[]
    link = link[link[:,1] <= tau]

    df = pd.DataFrame({"drought":count, "drought1":count})
    df.drought1 = df.drought1.shift(abs(tau))
    df = df.dropna()
    index = int(df.shape[0]*0.7)
    dim = df.shape[1]

    x_train, x_test = df.iloc[:index,1:dim], df.iloc[index:,1:dim] 
    y_train, y_test = df.iloc[:index,0], df.iloc[index:,0] 
    
    #model = RandomForestRegressor(max_depth=max_depth, random_state=0, n_estimators=n_estimators)
    #model = xgb.XGBRegressor(n_estimators=n_estimators,reg_lambda=1,gamma=0,max_depth=max_depth)
    model = CatBoostRegressor(depth=2)
    
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    result.append(mean_squared_error(y_pred, y_test))


    for z in range(len(link)):
        df = pd.DataFrame({"drought":count, "drought1":count})
        df.drought1 = df.drought1.shift(abs(tau))
        for k in range(0,z+1):
                if link[k,0] != 0:
                    df[str(link[k,0]-1)] = result_sst[:,link[k,0]-1]
                    df[str(link[k,0]-1)] = df[str(link[k,0]-1)].shift(abs(link[k,1]))
                else:
                    df[str(link[k,0])] = count
                    df[str(link[k,0])] = df[str(link[k,0])].shift(abs(link[k,1]))
        df = df.dropna()
        index = int(df.shape[0]*0.7)
        dim = df.shape[1]

        x_train, x_test = df.iloc[:index,1:dim], df.iloc[index:,1:dim] 
        y_train, y_test = df.iloc[:index,0], df.iloc[index:,0] 
        #model = RandomForestRegressor(max_depth=max_depth, random_state=0, n_estimators=n_estimators)
        #model = xgb.XGBRegressor(n_estimators=n_estimators,reg_lambda=1,gamma=0,max_depth=max_depth)
        model = CatBoostRegressor(iterations=2,learning_rate=1,depth=16)
        
        model.fit(x_train, y_train)
        y_pred = model.predict(x_test)
        result.append(mean_squared_error(y_pred, y_test))
    return(result,link)

In [4]:
def min_MSE_finder(count, result_sst, link,n_estimators=100, max_depth=5, tau=-1):
    result =[]
    link = link[link[:,1] <= tau]

    df = pd.DataFrame({"drought":count, "drought1":count})
    df.drought1 = df.drought1.shift(abs(tau))
    df = df.dropna()
    index = int(df.shape[0]*0.7)
    dim = df.shape[1]

    x_train, x_test = df.iloc[:index,1:dim], df.iloc[index:,1:dim] 
    y_train, y_test = df.iloc[:index,0], df.iloc[index:,0] 
    model = RandomForestRegressor(max_depth=max_depth, random_state=0, n_estimators=n_estimators)
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    result.append(mean_squared_error(y_pred, y_test))


    for z in range(len(link)):
        df = pd.DataFrame({"drought":count, "drought1":count})
        df.drought1 = df.drought1.shift(abs(tau))
        for k in range(0,z+1):
                if link[k,0] != 0:
                    df[str(link[k,0]-1)] = result_sst[:,link[k,0]-1]
                    df[str(link[k,0]-1)] = df[str(link[k,0]-1)].shift(abs(link[k,1]))
                else:
                    df[str(link[k,0])] = count
                    df[str(link[k,0])] = df[str(link[k,0])].shift(abs(link[k,1]))
        df = df.dropna()
        index = int(df.shape[0]*0.7)
        dim = df.shape[1]

        x_train, x_test = df.iloc[:index,1:dim], df.iloc[index:,1:dim] 
        y_train, y_test = df.iloc[:index,0], df.iloc[index:,0] 
        model = RandomForestRegressor(max_depth=max_depth, random_state=0, n_estimators=n_estimators)
        model.fit(x_train, y_train)
        y_pred = model.predict(x_test)
        result.append(mean_squared_error(y_pred, y_test))
    return(result,link)

In [5]:
def result_creator(name, code, missing, n_components, temporal_limits = temporal_limits):

    d = Data('{}'.format(name),code,temporal_limits,missing_value=missing)

    result = d.get_data()
    lon_list = d.get_lon_list()
    lat_list = d.get_lat_list()
    lon = d.get_lon()
    lat = d.get_lat()

    result = deseasonalize(np.array(result))
    weights = np.sqrt(np.abs(np.cos(np.array(lat_list)* math.pi/180)))
    for i in range(len(weights)):
        result[:,i] = weights[i] * result[:,i]

    data = pd.DataFrame(result)
    pca = PCA(n_components=n_components)
    pca_model = pca.fit(data)

    return(pca_model.transform(data))

In [5]:
results = load_obj("results_st")
q_matrix = np.load("q_matrix_sst.npy")

In [93]:
pq_matrix = results['p_matrix']
val_matrix = results['val_matrix']
alpha_level = 0.05
N = pq_matrix.shape[0]

link_dict = dict()
for j in range(N):
    # Get the good links
    good_links = np.argwhere(pq_matrix[:, j, 1:] <= alpha_level)
    # Build a dictionary from these links to their values
    links = {(i, -tau - 1): np.abs(val_matrix[i, j, abs(tau) + 1])
             for i, tau in good_links}
    # Sort by value
    link_dict[j] = sorted(links, key=links.get, reverse=True)

In [94]:
result_sst = result_creator("../../nc/sst.mnmean.nc","sst",-9.96921e+36,76)

In [95]:
link = np.array(link_dict[0])
if link[0,0] == 0:
    link = link[1:,: ]

In [96]:
n_estimators=100
max_depth=100
tau = -6

In [97]:
#np.random.shuffle(link[1:,:])
link_list = []
result, link = min_MSE_finder(count, result_sst, link,n_estimators, max_depth,tau)
link_list.append(link)
overall_min_MSE = []
overall_min_MSE.append(min(result))

In [98]:

diff = [x - result[i - 1] for i, x in enumerate(result)][1:]
I = np.array(diff) < 0 
while not all(I):
    link_list.append(link)
    link = link[I,:]
    result, link = min_MSE_finder(count, result_sst, link,n_estimators, max_depth,tau)
    overall_min_MSE.append(min(result))
    diff = [x - result[i - 1] for i, x in enumerate(result)][1:]
    I = np.array(diff) < 0 

In [99]:
i = np.argmin(np.array(overall_min_MSE))

In [89]:
link_list[i]

array([[  2, -12],
       [ 72, -11],
       [ 22,  -8],
       [ 54,  -6],
       [ 14,  -6],
       [ 29, -10],
       [  5,  -8],
       [ 70,  -6],
       [ 67,  -9],
       [ 52,  -7],
       [ 10,  -7],
       [ 33,  -8],
       [ 67,  -6]])

In [90]:
l = link_list[i ]

In [91]:
list(map(str,list(l[:,0])))

['2', '72', '22', '54', '14', '29', '5', '70', '67', '52', '10', '33', '67']

In [55]:
result_sst[:,l[:,0]-1].shape

(468, 8)

In [19]:
def link_finder1(count, results_name, q_matrix_name, file_name, code, missing,
                n_components, tau, n_estimators=100,max_depth=100):

    results = load_obj(results_name)
    q_matrix = np.load(q_matrix_name)

    pq_matrix = results['p_matrix']
    val_matrix = results['val_matrix']
    alpha_level = 0.05
    N = pq_matrix.shape[0]

    link_dict = dict()
    for j in range(N):
        # Get the good links
        good_links = np.argwhere(pq_matrix[:, j, 1:] <= alpha_level)
        # Build a dictionary from these links to their values
        links = {(i, -tau - 1): np.abs(val_matrix[i, j, abs(tau) + 1])
                 for i, tau in good_links}
        # Sort by value
        link_dict[j] = sorted(links, key=links.get, reverse=True)

    result_ = result_creator(file_name,code, missing, n_components)

    link = np.array(link_dict[0])
    if link[0,0] == 0:
        link = link[1:,: ]

    #np.random.shuffle(link[1:,:])

    result, link = min_MSE_finder1(count, result_, link,n_estimators, max_depth,tau)

    return(link, min(result))


In [6]:
def link_finder(count, results_name, q_matrix_name, file_name, code, missing,
                n_components, tau, n_estimators=100,max_depth=100):
    
    results = load_obj(results_name)
    q_matrix = np.load(q_matrix_name)

    pq_matrix = results['p_matrix']
    val_matrix = results['val_matrix']
    alpha_level = 0.05
    N = pq_matrix.shape[0]

    link_dict = dict()
    for j in range(N):
        # Get the good links
        good_links = np.argwhere(pq_matrix[:, j, 1:] <= alpha_level)
        # Build a dictionary from these links to their values
        links = {(i, -tau - 1): np.abs(val_matrix[i, j, abs(tau) + 1])
                 for i, tau in good_links}
        # Sort by value
        link_dict[j] = sorted(links, key=links.get, reverse=True)

    result_ = result_creator(file_name,code, missing, n_components)

    link = np.array(link_dict[0])
    if link[0,0] == 0:
        link = link[1:,: ]

    #np.random.shuffle(link[1:,:])
    link_list = []
    result, link = min_MSE_finder(count, result_, link,n_estimators, max_depth,tau)
    link_list.append(link)
    overall_min_MSE = []
    overall_min_MSE.append(min(result))


    diff = [x - result[i - 1] for i, x in enumerate(result)][1:]
    I = np.array(diff) < 0 
    while not all(I):
        link_list.append(link)
        link = link[I,:]
        result, link = min_MSE_finder(count, result_, link,n_estimators, max_depth,tau)
        overall_min_MSE.append(min(result))
        diff = [x - result[i - 1] for i, x in enumerate(result)][1:]
        I = np.array(diff) < 0 

    i = np.argmin(np.array(overall_min_MSE))

    return(link_list[i], overall_min_MSE)

In [128]:
best_link_sat, all_MSE_sat= link_finder(count,"results_sat","q_matrix_sat.npy","../../nc/air.mon.mean.nc","air",-9.96921e+36,82,-6)

In [100]:
best_link_sst, all_MSE_sst= link_finder(count,"results_st","q_matrix_sst.npy","../../nc/sst.mnmean.nc","sst",-9.96921e+36,76,-6)

In [22]:
result_list = []
for i in range(2):
    best_link_sst, all_MSE_sst= link_finder1(count,"results_st","q_matrix_sst.npy","../../nc/sst.mnmean.nc","sst"
                                            ,-9.96921e+36,76,-6)
    result_list.append(best_link_sst)

In [23]:
best_link_sst

array([[  2, -12],
       [ 22,  -8],
       [ 52, -10],
       [ 14,  -6],
       [  5,  -8],
       [ 39, -10],
       [ 49,  -6],
       [ 67, -12],
       [ 34, -11],
       [ 48, -10],
       [ 33,  -8]])

In [18]:
all_MSE_sst

885.5603021897809

In [14]:
lt = result_list[0].T
l = list(zip(lt[0],lt[1]))

for i in range(1,len(result_list)):
    lt = result_list[i].T
    lt = list(zip(lt[0],lt[1]))
    l = l + lt

In [15]:
result_dic = {}
for item in set(l):
    result_dic[item] = l.count(item)

In [16]:
result_dic

{(0, -12): 2,
 (0, -11): 2,
 (1, -10): 2,
 (2, -12): 2,
 (2, -11): 2,
 (5, -8): 2,
 (8, -12): 2,
 (10, -7): 2,
 (14, -6): 2,
 (17, -7): 2,
 (21, -7): 2,
 (22, -8): 2,
 (29, -10): 2,
 (33, -8): 2,
 (34, -11): 2,
 (38, -8): 2,
 (39, -10): 2,
 (43, -11): 2,
 (46, -11): 2,
 (48, -10): 2,
 (49, -6): 2,
 (52, -10): 2,
 (52, -7): 2,
 (54, -6): 2,
 (57, -6): 2,
 (67, -12): 2,
 (67, -9): 2,
 (67, -6): 2,
 (70, -6): 2,
 (71, -6): 2,
 (72, -11): 2,
 (72, -6): 2}

In [11]:
file_name = "./features/result_dic_{}.txtx".format(abs(tau))
save_obj(result_dic,file_name)

In [28]:
file_name = "./features/result_dic_{}.txt".format(abs(tau))

In [33]:
with open(file_name, 'w') as f:
    for key, values in result_dic.items():
        string = "{},{}\n".format(key,values)
        f.write(string)

In [22]:
for key, values in result_dic.items():
    string = "{},{}".format(key,values)
    print(string)

(33, -8),2
(52, -7),1
(22, -8),2
(67, -9),1
(54, -6),2
(72, -11),2
(2, -12),2
(5, -8),2
(10, -7),2
(29, -10),2
(48, -10),1
(14, -6),2
(38, -8),2
(67, -6),2


In [55]:
lt = result_list[0].T
l = list(zip(lt[0],lt[1]))

In [56]:
for i in range(1,len(result_list)):
    lt = result_list[i].T
    lt = list(zip(lt[0],lt[1]))
    l = l + lt

In [57]:
result_dic = {}
for item in set(l):
    result_dic[item] = l.count(item)

In [67]:
file_name = "./features/result_dic_{}".format(abs(tau))

In [68]:
file_name

'./features/result_dic_1'

In [59]:
save_obj(result_dic,"result_dic")

In [60]:
r = load_obj("result_dic")

{(2, -12): 10,
 (5, -8): 10,
 (10, -7): 5,
 (14, -6): 10,
 (22, -8): 10,
 (29, -10): 8,
 (33, -8): 10,
 (38, -8): 8,
 (48, -10): 1,
 (49, -6): 1,
 (52, -7): 7,
 (54, -6): 8,
 (67, -12): 1,
 (67, -9): 1,
 (67, -6): 5,
 (70, -6): 4,
 (71, -6): 3,
 (72, -11): 7,
 (72, -6): 5}

In [27]:
unique, counts = np.unique(result_list[0][:,:], return_counts=True)

In [32]:
l = result_list[0]

In [35]:
lt = l.T

In [40]:
 l = list(zip(lt[0],lt[1]))

In [43]:
l

[(2, -12),
 (72, -11),
 (22, -8),
 (54, -6),
 (14, -6),
 (29, -10),
 (5, -8),
 (49, -6),
 (33, -8),
 (71, -6)]

In [18]:
result_list[0][:,0]

array([ 2, 72, 22, 54, 14, 29,  5, 49, 33, 71])

In [78]:
tau = 12
file_name = './features/result_dic_{}.txt'.format(tau)
with open(file_name, 'r') as f:
    Lines = f.readlines()
    
result = {}
for Line in Lines:
    line = Line.strip().split(")")
    key = (int(line[0].split(',')[0][1:]), int(line[0].split(',')[1].strip()))
    value = int(line[1].strip(","))
    if value >= 475:
        result[key] = value
        

In [79]:
result

{(0, -12): 500, (2, -12): 500, (8, -12): 500, (67, -12): 500}

In [ ]:

file_name = './features/result_dic_{}_new.txt'.format(tau)
with open(file_name, 'r') as f:
    Lines = f.readlines()

result_new = {}
for Line in Lines:
    line = Line.strip().split(")")
    key = (int(line[0].split(',')[0][1:]), int(line[0].split(',')[1].strip()))
    value = int(line[1].strip(","))
    if value >= 475:
        result_new[key] = val

In [51]:
result_new

{(2, -12): 500, (22, -8): 500, (33, -8): 500}

In [16]:
line = Lines[0].strip().split(")")

In [41]:
(int(line[0].split(',')[0][1:]), int(line[0].split(',')[1].strip()))

(52, -10)

In [46]:
int(line[1].strip(","))

54

In [36]:
line = Lines[0].strip().split(")")

In [47]:
0.95 * 500

475.0

In [39]:
line[0].split(',')

['(52', ' -10']

In [15]:
tau = 2
file_name = './features/result_dic_{}_new.txt'.format(tau)
with open(file_name, 'r') as f:
    Lines = f.readlines()

result = {}
for Line in Lines:
    line = Line.strip().split(")")
    key = (int(line[0].split(',')[0][1:]), int(line[0].split(',')[1].strip()))
    value = int(line[1].strip(","))
    if value >= 475:
        result[key] = value

In [130]:
best_link_pre, all_MSE_pre= link_finder(count,"results_pre","q_matrix_pre.npy","../../nc/precip.mon.total.2.5x2.5.v2018.nc"
                                        ,"precip",-9.96921e+36,87,-6)

In [131]:
best_link_pres, all_MSE_pres= link_finder(count,"results_pres","q_matrix_pres.npy","../../nc/pres.mon.mean.nc"
                                        ,"pres",-9.96921e+36,64,-6)

In [142]:
np.mean(all_MSE_sst)

779.4454927392622